#### Steps
```plaintext
- Import the csv file into pandas df
- clean the table name and remove all extra symbols, spaces, capital letters
- clean the column header and remove all extra symbols, spaces, capital letters 
- write the create table SQL statement
- Import the data into the database

In [85]:
import pandas as pd
import numpy as np 
import os 
import psycopg2

In [86]:
ds = pd.read_csv('datasets/Customer Contracts$.csv')
ds.head()

,customer Name,start date,end date,contract amount ($M),invoice sent,Paid?
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


#### clear the table names

```plaintext
lower case letters
remove all the white spaces and $ 
replace -, /, \\, with _ 

In [87]:
file = 'Customer Contracts$'

clean_tbl_name = file.lower().replace(' ', '_').replace('?', '') \
                             .replace('-', '_').replace(r'/', '_').replace('\\', '_').replace('%', '') \
                             .replace(')', '').replace(r'(', '').replace('$', '')
                             
print(clean_tbl_name)

customer_contracts


In [88]:
ds.columns = [x.lower().replace(' ', '_').replace('?', '') \
                             .replace('-', '_').replace(r'/', '_').replace('\\', '_').replace('%', '') \
                             .replace(')', '').replace(r'(', '').replace('$', '') for x in ds.columns]
ds.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

SQL statement to create the table

```plaintext
create table customer_contract 
(
    customer_name       varchar
    start_date          varchar
    end_date            varchar
    contract_amount_m   float
    invoice_sent        varchar
    paid                varchar
);

The above Dtypes of the table do not match the dtypes of the pandas dtypes hence we have to match them

In [89]:
replacements = {
    'object'          : 'varchar',
    'float64'         : 'float',
    'int64'           : 'int',
    'datetime64'      : 'timestamp',
    'timedelta64[ns]' : 'varchar'
}

col_str = ', '.join('{} {}'.format(n, d) for (n, d) in zip(ds.columns, ds.dtypes.replace(replacements)))
col_str

'customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar'

```plaintext
Important to check - 

Go to settings and then to networking 
- Allow public access to this resource through the internet using a public IP address
- Allow public access from any Azure service within Azure to this server
- Add the rule - AllowAll 0.0.0.0 - 255.255.255.255

```plaintext
The below connection string can be obtained from - 

open the azure database
1) left hand side click on settings click connect 
2) click connect from your app and select the python option

In [90]:
try:
    conn = psycopg2.connect(user="*****", password="***********", host="dbserverpg.postgres.database.azure.com", port=5432, database="postgres")
    cursor = conn.cursor()
    print('Opened Database successfully')
except psycopg2.OperationalError as e:
    print(f"Error: {e}")

Opened Database successfully


In [91]:
# dropping tables with same name
cursor.execute('drop table if exists customer_contracts;')

In [92]:
#creating table
cursor.execute('create table customer_contracts \
               (customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, \
               invoice_sent varchar, paid varchar)')

In [93]:
# insert values to the table

# save df to csv
ds.to_csv('datasets/customer_contracts.csv', header=ds.columns, index=False, encoding='utf-8')

# we are opening the df in our ram as a Object 
my_file = open('datasets/customer_contracts.csv')
print('file opened in memory')

file opened in memory


In [94]:
# upload the object to the database
SQL_STATEMENT = '''
COPY customer_contracts FROM STDIN WITH 
    CSV 
    HEADER 
    DELIMITER AS ','
'''

cursor.copy_expert(sql=SQL_STATEMENT, file=my_file)
print('file copied to the db')

file copied to the db


In [95]:
cursor.execute('grant select on table customer_contracts to public')
conn.commit()

print('table customer_contract import to db completed')

table customer_contract import to db completed


---

In [96]:
cursor.execute("SELECT datname FROM pg_database;")
databases = cursor.fetchall()
print("Databases:")
for db in databases:
    print(f"- {db[0]}")

Databases:
- azure_maintenance
- template1
- postgres
- azure_sys
- template0


In [97]:
cursor.execute("""
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'public';
""")
tables = cursor.fetchall()

for table in tables:
    print(table[0])

customer_contracts


In [98]:
cursor.execute("SELECT * FROM customer_contracts;")
records = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]

column_widths = [max(len(str(record[i])) for record in records) for i in range(len(columns))]
column_widths = [max(len(columns[i]), column_widths[i]) for i in range(len(columns))]  # Include header lengths

header = " | ".join([columns[i].ljust(column_widths[i]) for i in range(len(columns))])
separator = "-" * len(header)

print(header)  
print(separator)  

for record in records:
    formatted_record = " | ".join([str(record[i]).ljust(column_widths[i]) if record[i] is not None else "None".ljust(column_widths[i]) for i in range(len(record))])
    print(formatted_record)

customer_name | start_date | end_date   | contract_amount_m | invoice_sent | paid
---------------------------------------------------------------------------------
Nike          | 01-02-2019 | 12-20-2020 | 2.98              | Yes          | Yes 
Reebox        | 06-20-2017 | None       | 3.9               | No           | No  
Adidas        | 12-07-2015 | 6-20-2018  | 4.82              | Yes          | Yes 
Google        | 05-25-2014 | 03-20-2017 | 5.74              | Yes          | No  
Amazon        | 11-10-2012 | 12-20-2015 | 6.66              | No           | Yes 
Facebook      | 04-29-2011 | None       | 7.58              | Yes          | No  
Apple         | 10-15-2009 | None       | 8.5               | Yes          | Yes 
Airbnb        | 04-02-2008 | None       | 9.42              | No           | No  
Nest          | 09-19-2006 | None       | 3.0               | Yes          | Yes 
Canon         | 03-07-2005 | 09-20-2009 | 11.26             | Yes          | No  


In [99]:
cursor.close()

### Connecting DBeaver to Azure PostgreSQL - Flexible Server

### Steps:
1. **Open DBeaver:** Launch the application.  
2. **New Connection:** Click "New Database Connection" or navigate to `Database > New Database Connection`.  
3. **Select PostgreSQL:** Choose "PostgreSQL" and click **Next**.  
4. **Configure Connection:**  
   - **Host:** `dbserverpg.postgres.database.azure.com`  
   - **Port:** `5432` (default) or `6432` (for PgBouncer)  
   - **Database:** Enter the target database (default: `postgres`)  
   - **Username:** `aariz`  
   - **Password:** Enter the admin password (check "Save password locally" if desired)  
5. **Advanced Settings:**  
   - Go to the **PostgreSQL** tab.  
   - Check **Show all databases** to display all databases.  
6. **Test Connection:** Click **Test Connection** to verify. Install drivers if prompted.  
7. **Finish:** Click **Finish** to save the connection.  
8. **Access Database:** Expand the server in the **Database Navigator** to browse and manage data.  

> **Note:** Ensure your client IP is allowed in Azure PostgreSQL firewall settings via the Azure portal under **Networking**.
